In [1]:
%run ../base_setup.ipynb

Foundation: 1.02.015
Transition: 2.03.026


In [2]:
from ds_behavioral.sample.sample_data import ProfileSample

In [3]:
builder = DataBuilder('synthetic_data_customer')
tools = builder.tools

### Build the Core Attributes

In [4]:
sample_size = 1000

df = tools.get_profiles(size=sample_size, mf_weighting=[5,3])
df['id'] = tools.unique_identifiers(from_value=1000000, to_value=9999999, prefix='CU_', size=sample_size)

value_distribution = [0.01,0.8,1,3,9,8,3,2,1] + list(np.flip(np.exp(np.arange(-5,0.0, 0.2)).round(2)))
df['balance'] = tools.get_number(0.0, 1000, precision=2, weight_pattern=value_distribution, size=sample_size)

age_pattern = [3,5,6,10,6,5,7,15,5,2,1,0.5,0.2,0.1]
df['age'] = tools.get_number(20.0, 90.0, weight_pattern=age_pattern, quantity=0.85, size=sample_size)

df['start'] = tools.get_datetime(start='01/01/2018', until='31/12/2018', date_format='%m-%d-%y', size=sample_size)

prof_pattern = [10,8,5,4,3,2] + [1]*9
profession = ProfileSample.professions(size=15)
df['profession'] = tools.get_category(selection=profession, weight_pattern=prof_pattern, quantity=0.90, size=sample_size)

df['online'] = tools.get_category(selection=[1,0], weight_pattern=[1,4], size=sample_size)


### Add in example 'Noise'

In [5]:
df['single num'] = tools.get_number(1, 1, quantity=0.8, size=sample_size, seed=31)
df['weight_num'] = tools.get_number(1, 2, weight_pattern=[90, 1], size=sample_size, seed=31)
df['null'] = tools.get_number(1, 100, quantity=0, size=sample_size, seed=31)
df['single cat'] = tools.get_category(['A'], quantity=0.6, size=sample_size, seed=31)
df['weight_cat'] = tools.get_category(['A', 'B', 'C'], weight_pattern=[80, 1, 1], size=sample_size, seed=31)


#### (optionally) Additional Extra Columns

In [6]:
if 'add_extra' in locals() and add_extra:
    df['last_login'] = tools.get_datetime(start='01/01/2019', until='01/05/2019', date_pattern=[1,2,3,5,9,20], date_format='%m-%d-%y %H:%M', size=sample_size)
    df['status'] = tools.get_category(selection=['Active', 'Closed', 'Suspended'], weight_pattern=[50,5,2], size=sample_size)

#### (optionally) Add volume of noise columns

In [7]:
if 'add_noise' in locals() and add_noise:
    for i in range(40):
        quantity = tools.get_number(0.005, 0.03, weight_pattern=[5,2,1,0.5])[0]
        col = "noise_{}".format(i)
        df[col] = tools.get_number(0, 1, weight_pattern=[20,1], quantity=quantity, size=sample_size)

### Save and Validate

In [11]:
filename = os.path.join(os.environ['DTU_ORIGIN_PATH'],'synthetic_customer.csv')
builder.save_to_disk(df, filename=filename)
discover.data_dictionary(df)

,Attribute,dType,%_Null,%_Dom,Count,Unique,Observations
0,age,float64,0.15,0.002,850,844,max=80.778 | min=20.035 | mean=47.36
1,balance,float64,0.00,0.002,1000,978,max=949.37 | min=30.46 | mean=183.87
2,forename,object,0.00,0.002,1000,994,Sample: Carlton | Lucinda | Felton
3,gender,category,0.00,0.606,1000,2,F|M
4,id,object,0.00,0.001,1000,1000,Sample: CU_8273488 | CU_5605291 | CU_6365281
5,null,object,1.00,0.000,0,0,Sample: Null Values
6,online,int64,0.00,0.798,1000,2,max=1 | min=0 | mean=0.2
7,profession,object,0.10,0.262,1000,16,Sample: Account Coordinator | Web Developer II | Cost Accountant
8,single cat,object,0.40,1.000,1000,2,Sample: | A
9,single num,float64,0.20,1.000,800,1,max=1.0 | min=1.0 | mean=1.0
